In [1]:
import descarteslabs as dl
import os
import sys
import json
import numpy as np
from shapely.geometry import mapping
from tqdm.notebook import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [38]:
roi_file = '../data/boundaries/dominican_republic.geojson'

In [39]:
with open(roi_file, 'r') as f:
    fc = json.load(f)
    geom = fc['features'][0]['geometry']

In [40]:
# grab dlkeys that intersect entire shape
tiles = dl.scenes.DLTile.from_shape(shape=geom,
                                    resolution=10,
                                    tilesize=900,
                                    pad=0)

print(f"Found {len(tiles)} over shape {os.path.basename(roi_file)}")

Found 699 over shape dominican_republic.geojson


In [41]:
# define population raster and threshold
pop_id = 'd15c019579fa0985f7006094bba7c7288f830e1f:GPW_Population_Density_V4_0'
pop_bands = ['population']
start_datetime = '2019-01-01'
end_datetime = '2021-01-01'
pop_thresh = 50

In [42]:
# filter dlkeys by population
tiles_filt = list()
for tile in tqdm(tiles):
    # rasterize population at 1km resolution
    aoi = dl.scenes.AOI(geometry=tile.geometry,
                        resolution=1000,
                        crs=tile.crs)
    
    scenes, ctx = dl.scenes.search(products=pop_id,
                                   aoi=aoi,
                                   start_datetime=start_datetime,
                                   end_datetime=end_datetime)
    
    if not scenes:
        continue
        
    pop = scenes.mosaic(bands=pop_bands, ctx=ctx)
    if np.any(pop > pop_thresh):
        tiles_filt.append(tile)

  0%|          | 0/699 [00:00<?, ?it/s]

In [43]:
features = list()
for t in tiles_filt:
    geom = mapping(t.geometry)
    feature = {'type': 'Feature',
               'geometry': geom}
    features.append(feature)

In [44]:
fc_out = {'type': 'FeatureCollection', 'features': features}

In [45]:
roi_out = 'dr_tiles_pop_filt_50.geojson'
with open(roi_out, 'w') as f:
    json.dump(fc_out, f)